In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial, fabs
np.set_printoptions(precision=6)

In [2]:
# вариант 1

L = 6
R = 8
Q = 5

Theta = np.array([
    [0, 0.4, 0, 0, 0.6, 0, 0],
    [0, 0, 0.7, 0, 0, 0, 0.3],
    [0, 0, 0, 0.5, 0, 0.5, 0],
    [0, 0, 0.2, 0, 0.8, 0, 0],
    [0, 0.6, 0, 0, 0, 0.4, 0],
    [0, 0, 0, 0.3, 0, 0, 0.7],
    [1, 0, 0, 0, 0, 0, 0]
])

lambda0 = 1
mu = [1.3, 1.2, 1.5, 1.1, 1.8, 2]
k = [1, 1, 1, 1, 1, 1]




In [3]:
def get_omega(L, Theta):
    omega = np.array([1 / (L + 1) for _ in range(L + 1)])
    for _ in range(100000):
        omega = omega.dot(Theta)
    return omega  

omega = get_omega(L, Theta)
print(omega, sum(omega))

[0.143725 0.161108 0.134392 0.108078 0.172697 0.136275 0.143725] 0.9999999999999989


In [4]:
def get_lambdas(L, lambda0, Theta):
    omega = get_omega(L, Theta)
    lambdas = [0] * L
    for i in range(1, len(omega)):
        lambdas[i - 1] = lambda0 * (omega[i] / omega[0])
    return lambdas

def get_tau(L, k, mu, lambda0, Theta, omega):
        lambdas = get_lambdas(L, lambda0, Theta) 
        
        psy = [0] * L
        for i in range(len(psy)):
            psy[i] = lambdas[i] / (k[i] * mu[i])
        
        P0 = [0] * L
        b = [0] * L
        h = [0] * L
        n = [0] * L
        u = [0] * L
        tau = 0

        for i in range(len(P0)):
            summa = 0
            for j in range(k[i]):
                summa += (k[i] * psy[i])**j / factorial(j)
            P0[i] = ((((k[i] * psy[i])**k[i]) / 
                          (factorial(k[i]) * (1 - psy[i])) ) + summa)**(-1)
            b[i] = P0[i] * (((k[i]**k[i]) * (psy[i]**(k[i] + 1))) / 
                                (factorial(k[i]) * ((1 - psy[i])**2)))
            h[i] = psy[i] * k[i]
            n[i] = b[i] + h[i]
            u[i] = n[i] / lambdas[i]
            tau += lambdas[i] * u[i]

        tau = (1 / lambda0) * tau
        return tau


## A) Вектор интенсивностей обслуживания задается произвольно. Необходимо подобрать такую маршрутную матрицу (при сохранении топологии), чтобы время реакции было минимальным:

In [ ]:
theta = np.zeros((L + 1, L + 1))
tau = float('inf')

enumeration = np.arange(0.1, 1, 0.15)
for s0 in enumeration:
    for s1 in enumeration:
        for s2 in enumeration:
            for s3 in enumeration:
                for s4 in enumeration:
                    for s5 in enumeration:
                        theta[0][1], theta[0][4] = s0, 1 - s0
                        theta[1][2], theta[1][6] = s1, 1 - s1
                        theta[2][3], theta[2][5] = s2, 1 - s2
                        theta[3][2], theta[3][4] = s3, 1 - s3
                        theta[4][1], theta[4][5] = s4, 1 - s4
                        theta[5][3], theta[5][6] = s5, 1 - s5
                        theta[6][0] = 1
#                         print(theta, '\n')
                        n_tau = get_tau(L, k, mu, lambda0, theta, omega)
                        if tau > n_tau and n_tau >= 0:
                            tau = n_tau
                            print(tau)
                            min_theta = theta.copy()

print('*' * 100)
print('Минимальное время реакции: ', tau)
print('при маршрутной матрице вида:\n', min_theta)

10.846872141209827
9.852954270454898
1.3473338624011504
1.201722854759145
0.7350806929547749
0.09849229403727389
0.09080164242041222
0.061628946978876
0.05703765754355894
0.012122614289082456
0.010918150980942842


## Б) Маршрутная матрица задается произвольно. Необходимо подобрать такой вектор интенсивностей обслуживания mu, чтобы сумма всех его компонент была минимальной, а пропускная способность сети была равна максимальной:

In [48]:
delta = 0.01

n_mu = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
lambdas = get_lambdas(L, lambda0, Theta)
for i in range(len(lambdas)):
    while lambdas[i] / n_mu[i] >= 1:
        n_mu[i] += delta
print('Вектор интенсивностей обслуживания\n', n_mu)
print('сумма элементов которого равна', sum(n_mu))
print('Максимальная пропусная способность сети: ', sum(lambdas))
    

Вектор интенсивностей обслуживания
 [1.1300000000000008, 0.9400000000000007, 0.7600000000000006, 1.2100000000000009, 0.9500000000000007, 1.0000000000000007]
сумма элементов которого равна 5.990000000000005
Максимальная пропусная способность сети:  5.957739380849532
